In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv("players_data.csv")
    
df.head()

,Unnamed: 0,name,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,cat_pos,position_DEF,position_FWD,position_GK,position_GKP,position_MID
0,0,Aaron Connolly,Brighton,2.1,0,0,-0.577658,0,-0.419363,-1.352937,...,0.196416,0.065203,-0.491967,-0.385999,FWD,0,1,0,0,0
1,1,Michail Antonio,West Ham,3.4,2,3,3.693568,0,1.423347,0.392776,...,-0.434511,-0.042959,0.213122,0.282991,FWD,0,1,0,0,0
2,2,Arthur Masuaku,West Ham,2.0,0,0,-0.577658,0,-0.419363,0.413254,...,-0.434511,-0.042959,0.213122,0.282991,DEF,1,0,0,0,0
3,3,Michael Olise,Crystal Palace,0.0,0,0,-0.577658,0,-0.419363,0.653865,...,-0.434511,0.065203,-0.726996,0.059994,MID,0,0,0,0,1
4,4,Michael Obafemi,Southampton,1.0,0,0,-0.577658,0,-0.419363,0.060015,...,-0.686881,-0.151120,-0.256937,-0.720494,FWD,0,1,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23194 entries, 0 to 23193
Data columns (total 55 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 23194 non-null  int64  
 1   name                       23194 non-null  object 
 2   team                       23194 non-null  object 
 3   xP                         23194 non-null  float64
 4   assists                    23194 non-null  int64  
 5   bonus                      23194 non-null  int64  
 6   bps                        23194 non-null  float64
 7   clean_sheets               23194 non-null  int64  
 8   creativity                 23194 non-null  float64
 9   element                    23194 non-null  float64
 10  fixture                    23194 non-null  float64
 11  goals_conceded             23194 non-null  int64  
 12  goals_scored               23194 non-null  int64  
 13  ict_index                  23194 non-null  flo

In [4]:
feature = df.drop(['Unnamed: 0','total_points', "team","GW","round"],axis = 1) 
target = df['total_points']
     

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.20, random_state=0)
     

X_train_name_value_pos=X_train[["name","value","cat_pos"]]
X_test_name_value_pos=X_test[["name","value","cat_pos"]]
     

X_train.drop(["name","value","cat_pos"],axis=1,inplace=True)
X_test.drop(["name","value","cat_pos"],axis=1,inplace=True)

# Model Implementation


In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_val_score

#param = {'kernel':['linear', 'poly', 'rbf'],"gamma":['scale', 'auto'] }
#model =  GridSearchCV(SVR(), param, cv=5)


model =  make_pipeline(StandardScaler(), SVR(kernel="rbf"))
model.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svr', SVR())])

In [8]:
#model.best_params_


In [9]:
prediction=model.predict(X_test)

In [10]:
prediction

array([0.06985527, 1.63204267, 1.81068079, ..., 0.10054112, 0.05171576,
       0.15288613])

In [11]:
mlr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': prediction})
mlr_diff.head(10)

,Actual value,Predicted value
4026,0,0.069855
5453,2,1.632043
22934,2,1.810681
22608,0,0.053594
438,0,0.018887
10568,1,0.092429
19825,6,1.061090
6033,1,0.083980
4194,0,0.072255
10220,0,0.074911


**model accuracy**

In [12]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, prediction), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, prediction), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, prediction), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, prediction), 2)) 
print("R2 score =", round(sm.r2_score(y_test, prediction), 2))

Mean absolute error = 0.95
Mean squared error = 4.57
Median absolute error = 0.11
Explain variance score = 0.26
R2 score = 0.23


In [13]:
prediction=np.round(prediction)
     

points_result = pd.DataFrame({"player name":X_test_name_value_pos["name"],
                              'Actual point': y_test, 
                              'Predicted point': prediction,
                              "player value":X_test_name_value_pos['value'],"position":X_test_name_value_pos["cat_pos"]})
points_result.to_csv("points_result_abdelrahman.csv")
     

points_result.head()

,player name,Actual point,Predicted point,player value,position
4026,Jarosław Jach,0,0.0,44,DEF
5453,Liam Cooper,2,2.0,45,DEF
22934,Adam Webster,2,2.0,43,DEF
22608,Louie Moulden,0,0.0,40,GK
438,James Justin,0,0.0,50,DEF
